## Importação das bibliotecas

In [1]:
# Importar as bibliotecas

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
import locale
import re
import os
from dotenv import load_dotenv

## Carregas as Variáveis de Ambiente

In [2]:
# Carregar variáveis do arquivo .env
load_dotenv()

# Obter as credenciais do banco de dados do .env
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [3]:
# Verifique se as variáveis foram carregadas corretamente
print(f"DB Host: {db_host}")
print(f"DB Port: {db_port}")
print(f"DB Name: {db_name}")
print(f"DB User: {db_user}")

DB Host: localhost
DB Port: 5432
DB Name: esquadrao
DB User: postgres


## Visualização inicial do DataFrame

In [4]:
# Caminho do arquivo Excel
excel_file = "dados\Export_Vendas_Sintetico.xlsx"

# Leitura do arquivo Excel e seleção das colunas desejadas
df = pd.read_excel(excel_file, sheet_name='Planilha1', usecols=['Data', 'Valor', 'Status', 'Operador', 'Vendedor'])

# Exibição o número de linhas e colunas do DataFrame
print("Número de Linhas e Colunas")
print(df.shape)
print("------------")

# Exibir as primeiras linhas do DataFrame
print("Primeiras Linhas do DataFrame")
print(df.head())
print("------------")

# Exibir os tipos de dados ajustados
print("Tipos de Dados")
print(df.dtypes)

Número de Linhas e Colunas
(103207, 5)
------------
Primeiras Linhas do DataFrame
                  Data    Valor     Status Operador Vendedor
0  21/12/2018 08:28:57   10.350  CANCELADA   Amanda     Neto
1  21/12/2018 09:16:04   10.350  CANCELADA     João   Victor
2  21/12/2018 09:16:02   10.350  CANCELADA     Taís    Lucas
3  21/12/2018 09:19:16    9.315  CANCELADA  Marcelo  Regiane
4  21/12/2018 09:23:14  224.850    REGULAR   Thales   Rebeca
------------
Tipos de Dados
Data         object
Valor       float64
Status       object
Operador     object
Vendedor     object
dtype: object


## Ajuste dos dados

In [ ]:
# Renomear as colunas
df.rename(columns={
    'Data': 'data',
    'Valor': 'valor',
    'Status': 'status',
    'Operador': 'operador',
    'Vendedor': 'vendedor'
}, inplace=True)

# Ajustar as variáveis da coluna status
df['status'] = df['status'].str.capitalize()

# Definir o locale para Português do Brasil (pt_BR)
locale.setlocale(locale.LC_NUMERIC, 'pt_BR.UTF-8')

# Função para converter valores
def convert_to_float(value):
    if isinstance(value, str):
        value = value.replace('.', '').replace(',', '.')
    return float(value)

# Aplicar a função de conversão na coluna 'valor' para float aplicando
df['valor'] = df['valor'].apply(convert_to_float)

# Converter a coluna 'data' para Timestamp
try:
    df['data'] = pd.to_datetime(df['data'])
except ValueError as e:
    print(f"Não foi possível converter a coluna 'data' para Timestamp: {e}")

# Truncar a coluna 'status' para 10 caracteres
df['status'] = df['status'].apply(lambda x: x[:10])

# Remover linhas onde o status é 'Cancelada' ou 'Inutilizada'
df = df[~df['status'].isin(['Cancelada', 'Inutilizad'])]

# Exibir as primeiras linhas do DataFrame após a conversão
print("Primeiras Linhas do DataFrame - compras devidamente efetivadas")
print(df.head())
print("------------")

# Exibir os tipos de dados ajustados
print("Tipos de Dados")
print(df.dtypes)

## Conexão com o Banco de Dados

In [ ]:
# Criar a tabela no banco de dados e inserir os dados
try:
    # Definir a string de conexão com o banco de dados PostgreSQL
    postgres_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
    engine = create_engine(postgres_str)

    with engine.connect() as connection:
        print("Conexão com o banco de dados estabelecida com sucesso!")

        # Nome do esquema e da tabela no banco de dados
        schema_name = 'esquadrao'
        table_name = 'tbl_loja'

        # Criar a tabela no banco de dados
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {schema_name}.{table_name} (
            data DATE,
            valor FLOAT,
            status VARCHAR(10),
            operador VARCHAR(255),
            vendedor VARCHAR(255)
        );
        """
        connection.execute(create_table_query)
        print(f"Tabela {schema_name}.{table_name} criada com sucesso!")

        # Limpar a tabela no banco de dados
        clear_table_query = f"TRUNCATE TABLE {schema_name}.{table_name};"
        connection.execute(clear_table_query)
        print(f"Tabela {schema_name}.{table_name} limpa com sucesso!")

        # Inserir os dados no banco de dados
        df.to_sql(table_name, engine, schema=schema_name, if_exists='replace', index=False)
        print("Dados inseridos no banco de dados com sucesso!")

except OperationalError as oe:
    print(f"Erro de conexão com o banco de dados: {oe}")
except Exception as e:
    print(f"Erro ao criar a tabela ou inserir dados no banco de dados: {e}")